# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [11]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [18]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
fiducial_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-17 - Mouse 210\Fiducial')
mouse_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-17 - Mouse 210\For registration')

registered_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-17 - Mouse 210\Registered')

In [5]:
microscopy_path = Path(mouse_dir, 'SHG down 8x.tif')
microscopy_tile_path = Path(mouse_dir, 'SHG settings.ome.tif')

us_path = Path(mouse_dir, 'L38 down2x 75dB.tif')
us_pl_path = Path(mouse_dir, 'L38 Mouse Z13046.pos')
us_param_path = Path(mouse_dir, 'L38 Mouse Z13046_Run-1_Settings.mat')

transform_path = Path(fiducial_dir, 'L38 Transform.txt')

Now open the image.  If the spacing information is not properly written to either image it can still be set manually.

In [27]:
spacing = [24, 24, 24]
gauge_height = 13046
us_image = coord.open_us(us_path, us_pl_path, us_param_path, spacing, gauge_height)

In [7]:
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=8)

Load the coordinate transform

In [8]:
with open(transform_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)

Apply the registration and save the results

In [23]:
mpm_reg_path = Path(registered_dir, 'Mouse 210 - L38 and SHG.tif')

In [29]:
mpm_reg = tran.apply_transform_params(us_image, microscopy_image, 
                                     -1*coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [30]:
ome.save_ijstyle_overlay([mpm_reg, us_image], mpm_reg_path)